In [ ]:
!pip install telethon
!pip install pandas

In [ ]:
rm session_name.session

rm: cannot remove 'session_name.session': No such file or directory


## SCRAPE USER FROM GROUPS ON TELEGRAMM

In [ ]:
from telethon import TelegramClient
from telethon.errors import FloodWaitError
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch, MessageEntityUrl
import pandas as pd
import asyncio
import time
import re

# Telegram API Credentials
API_ID = '******'
API_HASH = '******5'
GROUP_USERNAME = '*******'

client = TelegramClient('session_name', API_ID, API_HASH)

async def fetch_all_members(chat):
    members = []
    offset = 0
    limit = 100
    while True:
        try:
            participants = await client(GetParticipantsRequest(
                channel=chat,
                filter=ChannelParticipantsSearch(''),
                offset=offset,
                limit=limit,
                hash=0
            ))
        except FloodWaitError as e:
            print(f"FloodWaitError: waiting for {e.seconds}s")
            await asyncio.sleep(e.seconds)
            continue

        if not participants.users:
            break

        for user in participants.users:
            members.append({
                "User ID": user.id,
                "Username": user.username or "",
                "First Name": user.first_name or "",
                "Last Name": user.last_name or "",
                "Phone": user.phone or '',
                "Access Hash": user.access_hash,

            })

        offset += len(participants.users)
        if len(participants.users) < limit:
            break
    return members

async def fetch_all_messages(chat, limit=0):
    messages = []
    async for msg in client.iter_messages(chat, limit=limit):
        messages.append({
            "Date": msg.date,
            "Sender ID": msg.sender_id,
            "Message": msg.text.strip().replace('\n', ' ') if msg.text else "",
            "Message ID": msg.id
        })
    return messages

async def main():
    await client.start()
    chat = await client.get_entity(GROUP_USERNAME)



    print("[*] Fetching members...")
    members = await fetch_all_members(chat)
    members_df = pd.DataFrame(members)

    safe_title = re.sub(r'\W+', '_', chat.title)

    members_df.to_csv(f"group_members_{safe_title}.csv", index=False, encoding="utf-8")
    print(f"[+] Saved {len(members)} members to group_members_{safe_title}.csv")


    print("[*] Fetching messages...")
    messages = await fetch_all_messages(chat)
    messages_df = pd.DataFrame(messages)
    messages_df.to_csv("group_messages.csv", index=False, encoding="utf-8")
    print(f"[+] Saved {len(messages)} messages to group_messages.csv")

    await client.disconnect()

await main()


## GET USER IDS from Single USERS on TELEGRAMM

In [ ]:
from telethon import TelegramClient
import pandas as pd
import asyncio

API_ID = '+++++'
API_HASH = '++++++++++++++++++'

# Your list
USERNAMES = [
    'username1', 'username2', 'username3', 'username4', 'username5',
    'username6', 'username7', 'username8', 'username9', 'username10
]

client = TelegramClient('resolve_session', API_ID, API_HASH)

async def resolve_usernames(usernames):
    results = []
    for name in usernames:
        username = name.lstrip('@')
        try:
            user = await client.get_entity(username)
            results.append({
                "Input": name,
                "Username": user.username or '',
                "User ID": user.id,
                "First Name": user.first_name or '',
                "Last Name": user.last_name or '',
                "Phone": user.phone or '',
                "Access Hash": user.access_hash
            })
        except Exception as e:
            results.append({
                "Input": name,
                "Username": username,
                "User ID": "NO ID",
                "First Name": '',
                "Last Name": '',
                "Phone": '',
                "Error": str(e)
            })
    return results

async def main():
    await client.start()
    data = await resolve_usernames(USERNAMES)
    df = pd.DataFrame(data)
    df.to_csv("resolved_users.csv", index=False, encoding="utf-8")
    print(f"[+] Done. Saved {len(data)} results to resolved_users.csv")
    await client.disconnect()

await main()


[+] Done. Saved 5 results to resolved_users.csv


##SEARCH FOR GROUPS ON TELEGRAM

In [ ]:
#sucht nach gruppen name

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.contacts import SearchRequest
import asyncio
import os

API_ID = ##############
API_HASH = '##############'
SEARCH_KEYWORD = "###########"

# Start mit Standard-Session
SESSION_BASE = "search_session"
client = None

async def safe_create_client():
    global client
    for i in range(5):
        session_name = f"{SESSION_BASE}_{i}" if i > 0 else SESSION_BASE
        try:
            client = TelegramClient(session_name, API_ID, API_HASH)
            await client.start()
            print(f"[+] Connected using session: {session_name}")
            return
        except Exception as e:
            if "database is locked" in str(e).lower():
                print(f"[!] Session '{session_name}' is locked, trying another...")
                continue
            else:
                raise e
    raise RuntimeError("❌ Failed to start any TelegramClient session.")

async def search_groups_by_title(keyword):
    print(f"[*] Searching for groups/channels with keyword: '{keyword}'")
    results = []
    try:
        result = await client(SearchRequest(
            q=keyword,
            limit=30
        ))

        if not result.chats:
            print("[-] Keine Gruppen oder Kanäle gefunden.")
            return results

        for chat in result.chats:
            title = chat.title
            username = chat.username or "(privat oder leer)"
            chat_id = chat.id
            link = f"https://t.me/{chat.username}" if chat.username else "(kein öffentlicher Link)"

            print(" Titel     :", title)
            print(" Username :", f"@{username}")
            print(" ID        :", chat_id)
            print(" Link      :", link)
            print("-" * 40)

            results.append({
                "Title": title,
                "Username": username,
                "ID": chat_id,
                "Link": link
            })

    except Exception as e:
        print(f"[!] Fehler bei der Suche: {e}")
    return results

async def main():
    await safe_create_client()
    results = await search_groups_by_title(SEARCH_KEYWORD)

    if results:
        filename = f"search_results_{SEARCH_KEYWORD.replace(' ', '_')}.csv"
        pd.DataFrame(results).to_csv(filename, index=False, encoding="utf-8")
        print(f"[+] CSV gespeichert: {filename}")
    else:
        print("[-] Keine Daten zum Speichern.")

    await client.disconnect()

#  Start
await main()